<div style ="padding: 15px 20px;
    color: #c8102e;
    background: #01216955;             
    display: flex;
    justify-content: space-between;
    align-items: center;">
    <div>
        <img src="flag_mini.png">
    </div>
    <div>
        <h2 style = "padding : 0 10px">7 ) Afficher une carte de chaleur qui représente le nombre d'athlètes engagés par sport (en vertical) au cours du temps (en horizontal)</h2>
    </div>
</div>

In [21]:
import pandas as pd
import os
import altair as alt

country = 'GBR'

df_results = pd.read_csv(r"{}/archive/olympic_results.csv".format(os.getcwd()))
df_hosts = pd.read_csv(r"{}/archive/olympic_hosts.csv".format(os.getcwd()))

## Récupération de tous les participants anglais aux Jo d'été 

In [22]:
#fusionner df_results & df_hosts pour connaître les Jo d'étés
df = df_results.merge(df_hosts, left_on='slug_game', right_on='game_slug')

#filtrer : on garde les athlètes de GB médaillés au jeux d'étés
df.query("country_3_letter_code == @country and game_season == 'Summer'", inplace=True)
print('Pour cette étude, nous gardons un total de {} athletes ou équipes inscrites au JO.'.format(df.shape[0]))

Pour cette étude, nous gardons un total de 5300 athletes ou équipes inscrites au JO.


## Renommage des disciplines
**Certains nom de discipline ont changé au cours des années mais correspondent pourtant à la même compétition. Nous procédons aux renommage de celles-ci pour éviter des lignes doublées sur la heatmap**

In [23]:
df_shorted = df.copy()
# replaces changed disciplines names
discipline_map={
    "Gymnastics Artistic":"Artistic Gymnastics",
    "Gymnastics Rhythmic":"Rhythmic Gymnastics",
    "Synchronized Swimming":"Artistic Swimming",
    "Equestrian Dressage":"Equestrian",
    "Equestrian Jumping":"Equestrian",
    "Equestrian Eventing":"Equestrian",
    "Trampoline":"Trampoline Gymnastics",
    "Cycling BMX":"Cycling BMX Racing",
    "Short Track Speed Skating": "Short Track"
}
df_shorted['discipline_title'] = df_shorted['discipline_title'].replace(discipline_map)

## Initialisation du graphique ( heatmap )

In [24]:
source = df_shorted

# Configure common options
base = alt.Chart(source).transform_aggregate(
    participants='count()',
    groupby=['game_year', 'discipline_title']
).encode(
    alt.X('game_year:O', scale=alt.Scale(paddingInner=0), title="Années"),
    alt.Y('discipline_title:O', scale=alt.Scale(paddingInner=0), title="Disciplines"),
).properties(title = "Répartition des athlètes par discipline selon l'année")

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('participants:Q',
        scale=alt.Scale(scheme='viridis'),
        legend=alt.Legend(direction='horizontal')
    )
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    text='participants:Q',
    color=alt.condition(
        alt.datum.participants > 100,
        alt.value('black'),
        alt.value('white')
    )
)

# Changer le nombre de ligne à intégrer ( par défaut = 5000 )
alt.data_transformers.enable('default', max_rows=5500)

DataTransformerRegistry.enable('default')

# Résultats : 

In [25]:
heatmap + text

alt.LayerChart(...)